# **<font color="red">Agents Workflow</font>**
- Specialized agents that control the execution flow of its sub-agents.
1. `Sequential Agent`
2. `Loop Agent`
3. `Parallel Agent`


## **<font color="blue">Loop Agents</font>**
The `LoopAgent` is a workflow agent that executes its sub-agents in a loop (i.e., iteratively). It ***repeatedly runs a sequence of agents*** for a specified number of iterations or until a termination condition is met.
Use the `LoopAgent` when your workflow involves repetition or iterative refinement such as revising code.

In [8]:
"""
JOB DESCRIPTION OPTIMIZER
"""

import os
import asyncio

from google.adk.agents import LoopAgent, LlmAgent, SequentialAgent
from google.adk.tools.tool_context import ToolContext
from google.adk.agents.callback_context import CallbackContext
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from google.adk.models import Gemini


# ============================================================
# CONFIG
# ============================================================

from config import config
os.environ["GOOGLE_API_KEY"] = config.GOOGLE_API_KEY

MODEL = Gemini(model="gemini-2.5-flash")

APP_NAME = "jd_optimizer_pipeline"
USER_ID = "recruiter_1"
SESSION_ID = "session_jd_001"

STATE_JD_BRIEF   = "jd_brief"
STATE_CURRENT_JD = "current_jd"
STATE_CRITIQUE   = "critique"

COMPLETION_PHRASE = "JD_APPROVED"


# ============================================================
# TOOL — exit_loop
# ============================================================

def exit_loop(tool_context: ToolContext) -> dict:
    print(f"\n  🔔  exit_loop triggered by {tool_context.agent_name}")
    tool_context.actions.escalate = True
    tool_context.actions.skip_summarization = True
    return {}


# ============================================================
# CALLBACK
# ============================================================

def log_pipeline_start(callback_context: CallbackContext) -> None:
    brief = callback_context.state.get(STATE_JD_BRIEF)
    if brief:
        print("\n  ℹ️  JD brief loaded successfully.")


# ============================================================
# STAGE 1 — Draft Writer
# ============================================================

draft_writer_agent = LlmAgent(
    name="DraftWriterAgent",
    model=MODEL,
    include_contents="none",
    instruction="""
Write a professional Job Description from this brief:

{jd_brief}

Include exactly 7 sections:
1. Job Title & Location
2. About the Company
3. Role Overview
4. Key Responsibilities
5. Required Skills & Experience
6. Nice-to-Have Skills
7. Compensation & Benefits

Output ONLY the JD.
""",
    output_key=STATE_CURRENT_JD,
)


# ============================================================
# STAGE 2a — Critic
# ============================================================

jd_critic_agent = LlmAgent(
    name="JDCriticAgent",
    model=MODEL,
    include_contents="none",
    instruction=f"""
Review this Job Description:

{{current_jd}}

If it satisfies:
- Clarity
- Inclusivity
- Completeness
- Candidate Appeal

Output exactly:
{COMPLETION_PHRASE}

Otherwise output numbered improvement points only.
""",
    output_key=STATE_CRITIQUE,
)


# ============================================================
# STAGE 2b — Refiner (NO output_key intentionally)
# ============================================================

jd_refiner_agent = LlmAgent(
    name="JDRefinerAgent",
    model=MODEL,
    include_contents="none",
    tools=[exit_loop],
    instruction=f"""
Current JD:
{{current_jd}}

Critique:
{{critique}}

If critique is EXACTLY "{COMPLETION_PHRASE}":
    Call exit_loop tool.
    Output NOTHING.

Otherwise:
    Rewrite the FULL improved JD.
    Output ONLY the improved JD.
""",
)


# ============================================================
# LOOP AGENT
# ============================================================

refinement_loop = LoopAgent(
    name="JDRefinementLoop",
    sub_agents=[jd_critic_agent, jd_refiner_agent],
    max_iterations=5,
)


# ============================================================
# ROOT PIPELINE
# ============================================================

root_agent = SequentialAgent(
    name="JDOptimizerPipeline",
    sub_agents=[draft_writer_agent, refinement_loop],
    before_agent_callback=log_pipeline_start,
)


# ============================================================
# SESSION + RUNNER
# ============================================================

session_service = InMemorySessionService()

runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
)


# ============================================================
# PIPELINE EXECUTION
# ============================================================

async def run_jd_pipeline(job_brief: str):

    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={STATE_JD_BRIEF: job_brief},
    )

    user_message = types.Content(
        role="user",
        parts=[types.Part(text="Optimize this JD")],
    )

    # Process events safely
    for event in runner.run(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=user_message,
    ):

        # Only handle refiner responses
        if getattr(event, "author", "") != "JDRefinerAgent":
            continue

        content = getattr(event, "content", None)
        if not content or not hasattr(content, "parts"):
            continue

        extracted_text = None

        for part in content.parts:
            text = getattr(part, "text", None)

            if (
                text
                and isinstance(text, str)
                and text.strip()
                and text.strip() != COMPLETION_PHRASE
            ):
                extracted_text = text.strip()
                break

        if not extracted_text:
            continue

        # Persist improved JD
        session = await session_service.get_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID,
        )

        session.state[STATE_CURRENT_JD] = extracted_text

    # Final state read
    session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
    )

    final_jd = session.state.get(STATE_CURRENT_JD, "").strip()
    last_critique = session.state.get(STATE_CRITIQUE, "").strip()

    return final_jd, last_critique


# ============================================================
# MAIN
# ============================================================

async def main():

    job_brief = "Senior Data Engineer at GreenPay FinTech startup..."

    final_jd, last_critique = await run_jd_pipeline(job_brief)

    with open("optimized_jd.md", "w", encoding="utf-8") as f:
        f.write("# Optimized Job Description\n\n")
        f.write(final_jd)

    print("\n✅ Final JD saved correctly.")
    print("Critic Verdict:", last_critique)


if __name__ == "__main__":
    # asyncio.run(main())
    await main()




  ℹ️  JD brief loaded successfully.

  🔔  exit_loop triggered by JDRefinerAgent

✅ Final JD saved correctly.
Critic Verdict: JD_APPROVED
